### Data preparation for the Unserious Data: JHU in Film session for Love Data Week 2026

During this workshop, we will explore JHU in film by combining data from separate sources, including data on IMDB titles, data on IMDB ratings, and JHU titles, as described on the Hub article [Johns Hopkins on Film](https://hub.jhu.edu/2015/10/13/johns-hopkins-on-film/).


Given the time limits of the workshop, we will provide a partially precleaned version of these data for the session, to make the data easier to work with. If you are interested in learning how the data was prepared from it's raw, original source, please follow along below! 

### Scraping Film Titles from Hub Article
The titles JHU appears in are listed on the Hub article [Johns Hopkins on Film](https://hub.jhu.edu/2015/10/13/johns-hopkins-on-film/). We would like to structure these neatly as tabular data, with a column for the title, and a column for the year it was produced. We could manually transcribe the titles – there's not that many – but what's the fun in that?

Instead we can write a web scraper to extract the relevant information using the `Beautiful Soup` library. 


Let's import the libraries we need for scraping: `requests` to extract the HTML from the Hub article, `Beautiful Soup` to make sense of it, the regular expression library `re` to help pull out what we need, and `pandas` to format it as tabular data.


In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

Now let's formulate our request that extracts the HTML from the Hub article. That HTML will be hard to work with, so we can use the `Beautiful Soup` html parser to turn it into a parse tree, which makes it (relatively) easy to navigate the nested information in that Hub article.

In [10]:
response = requests.get('https://hub.jhu.edu/2015/10/13/johns-hopkins-on-film/')
soup = BeautifulSoup(response.content, 'html.parser')

When scraping, a good strategy is to identify any commonalities in the information you would like to extract. Let's take a look at the Hub article:
![Hub article](images/hopkins-on-film.png)

We see that "Filmed at Hopkins" is larger than the rest of the text – that's a heading. 

We then see that each of the titles are bolded. Nothing else on the page is bolded. This makes things easy! We just need to find the bolded text! We bold text in HTML using `<strong>I'm so bold!</strong>`, so to start, we need to find all the strong 💪 text.

In [ ]:
titles_raw = soup.find_all('strong')
titles_cleaned = []
for title in titles_raw:
    title_name_element = title.find('em')
    title_name = title_name_element.getText(strip=True)
    title_name_element.extract()
    title_year_raw = title.get_text(strip=True)
    title_year = re.search(r'(?<=\()(\d*)', title_year_raw).group(0)
    titles_cleaned.append({'Title': title_name, 'Year': title_year})

Now let's change that list to a `pandas` dataframe and save as a csv. This is the cleaned data you worked with during the workshop.

In [12]:
titles_data = pd.DataFrame(titles_cleaned) 
titles_data.to_csv('data/cleaned/jhu-titles.csv')

### Preparing IMDB Data
Now it's time to tackle the raw IMDB data. Let's take a look at the files:

In [15]:
%ls -lh data/title.*

-rw-r--r--@ 1 plawson  staff   2.5G Nov 20 20:36 data/title.akas.tsv
-rw-r--r--@ 1 plawson  staff   996M Nov 20 20:34 data/title.basics.tsv
-rw-r--r--@ 1 plawson  staff   380M Nov 20 20:34 data/title.crew.tsv
-rw-r--r--@ 1 plawson  staff   233M Nov 20 20:34 data/title.episode.tsv
-rw-r--r--@ 1 plawson  staff   4.0G Nov 20 20:36 data/title.principals.tsv
-rw-r--r--@ 1 plawson  staff    27M Nov 20 20:33 data/title.ratings.tsv


Yikes! Some of those files are pretty big! We could use `pandas` to load them in to a dataframe, but that's going to eat up a lot of memory. A smart way to work with the data would be to only load into memory exactly what we need, and nothing more. 

There is a great tool for doing this – databases! And Python has a library that make creating and working with databases really easy – `duckdb`. 

> Fun fact: Hannes Mühleise, the creator of `duckdb`, named it for his pet duck Wilbur, who used to live on a boat with him. So if you get intimidated working with databases, just remember Wilbur, and you will feel better! 

In [17]:
import duckdb

# open a duckdb database connection
con = duckdb.connect()

First, I want to return all the IMDB titles that match our JHU titles.

In [18]:
jhu_titles_imdb = con.execute(
    """
    SELECT tconst, titleType, primaryTitle, startYear, endYear, runtimeMinutes, genres
    FROM read_csv_auto(
    "./data/title.basics.tsv", 
    header=True,
    nullstr="\\N") as tb
    JOIN titles_data as td ON tb.originalTitle = td.Title
    WHERE tb.startYear = td.Year
    """
).fetchdf()

It would be too easy if I gave you just the matches for our workshop, so I will generate a smaller, more maneagable set of data to work with:

In [19]:
random_titles = con.execute(
    """
    SELECT *
    FROM (
        SELECT tconst, titleType, primaryTitle, startYear, endYear, runtimeMinutes, genres
        FROM read_csv_auto(
        "./data/title.basics.tsv",
        header=True,
        nullstr="\\N")
    WHERE isAdult = 0)
    USING SAMPLE 1000;
    """
).fetchdf()

Now let's combine our matches, and our random rows, and mix 'em up!

In [20]:
imdb_title_basics_subset = pd.concat([random_titles, jhu_titles_imdb]).sample(frac=1, random_state=42)
imdb_title_basics_subset.to_csv('data/cleaned/imdb-title-basics.csv', index=False)

Finally, let's read in the ratings data, and save as a .csv for consistency:

In [21]:
imdb_ratings = pd.read_csv('data/title.ratings.tsv', sep='\t')
imdb_ratings.to_csv('data/cleaned/imdb-ratings.csv', index=False)